embedding And vectorStoreDB


In [2]:
!pip3 install langchain_community pypdf


In [5]:
from langchain_community.document_loaders import PyPDFLoader

# Path to your PDF file (local)
loader = PyPDFLoader("/Users/vrushalipatil/Downloads/Revati_Resume.pdf")

# Load PDF into documents
docs = loader.load()
print(docs)


[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Bonish Agarwal - Resume', 'source': '/Users/vrushalipatil/Downloads/Revati_Resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Revati  Patil  MCA  graduate  with  strong  knowledge  of  frontend  and  backend  development,  eager  to  apply  \ntechnical\n \nskills\n \nto\n \nreal-world\n \nprojects.\n \n revatip290@gmail.com   +91  9579412102  g i t h u b . c o m / R e v a t i v P a t i l   l i n k e d i n . c o m / i n / r e v a t i - v - p a t i l /  \nWORK  EXPERIENCE Humming  Byte  Pvt.Ltd Web  Development  Intern   June  2023  -  Present •  Developed  and  maintained  responsive  full-stack  web  applications  using  Node.js ,  React.js ,  and  modern  \nframeworks.\n •  Integrated  Firebase  backend  services,  implementing  secure  authentication  and  real-time  database  \nfunctionalities.\n •  Enhanced  UI/UX  with  CSS ,  and  optimi

In [6]:
print(f"✅ Loaded {len(docs)} pages from PDF")
print(docs[0])

✅ Loaded 1 pages from PDF
page_content='Revati  Patil  MCA  graduate  with  strong  knowledge  of  frontend  and  backend  development,  eager  to  apply  
technical
 
skills
 
to
 
real-world
 
projects.
 
 revatip290@gmail.com   +91  9579412102  g i t h u b . c o m / R e v a t i v P a t i l   l i n k e d i n . c o m / i n / r e v a t i - v - p a t i l /  
WORK  EXPERIENCE Humming  Byte  Pvt.Ltd Web  Development  Intern   June  2023  -  Present •  Developed  and  maintained  responsive  full-stack  web  applications  using  Node.js ,  React.js ,  and  modern  
frameworks.
 •  Integrated  Firebase  backend  services,  implementing  secure  authentication  and  real-time  database  
functionalities.
 •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  performance  to  improve  application  eﬃciency  by  
reducing
 
response
 
time.
  EDUCATION MES  Abasaheb  Garware  College,  Pune Master  of  Computer  Application  (MCA)  -  7.16/  10  Aug  2023  -  May  2025  B.S.C  Computer  

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(f"✅ Created {len(splits)} chunks")

✅ Created 3 chunks


In [8]:
print(splits[0])
print(splits[1])
print(splits[2])

page_content='Revati  Patil  MCA  graduate  with  strong  knowledge  of  frontend  and  backend  development,  eager  to  apply  
technical
 
skills
 
to
 
real-world
 
projects.
 
 revatip290@gmail.com   +91  9579412102  g i t h u b . c o m / R e v a t i v P a t i l   l i n k e d i n . c o m / i n / r e v a t i - v - p a t i l /  
WORK  EXPERIENCE Humming  Byte  Pvt.Ltd Web  Development  Intern   June  2023  -  Present •  Developed  and  maintained  responsive  full-stack  web  applications  using  Node.js ,  React.js ,  and  modern  
frameworks.
 •  Integrated  Firebase  backend  services,  implementing  secure  authentication  and  real-time  database  
functionalities.
 •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  performance  to  improve  application  eﬃciency  by  
reducing
 
response
 
time.' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Bonish Agarwal - Resume', 'source': '/Users/vrushalipatil/Downlo

In [9]:
print(len(splits))

3


In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [11]:
texts = [doc.page_content for doc in splits]
print(texts[:2])

['Revati  Patil  MCA  graduate  with  strong  knowledge  of  frontend  and  backend  development,  eager  to  apply  \ntechnical\n \nskills\n \nto\n \nreal-world\n \nprojects.\n \n revatip290@gmail.com   +91  9579412102  g i t h u b . c o m / R e v a t i v P a t i l   l i n k e d i n . c o m / i n / r e v a t i - v - p a t i l /  \nWORK  EXPERIENCE Humming  Byte  Pvt.Ltd Web  Development  Intern   June  2023  -  Present •  Developed  and  maintained  responsive  full-stack  web  applications  using  Node.js ,  React.js ,  and  modern  \nframeworks.\n •  Integrated  Firebase  backend  services,  implementing  secure  authentication  and  real-time  database  \nfunctionalities.\n •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  performance  to  improve  application  eﬃciency  by  \nreducing\n \nresponse\n \ntime.', 'functionalities.\n •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  performance  to  improve  application  eﬃciency  by  \nreducing\n \nresponse\n \ntim

In [12]:
embeddings = model.encode(texts, convert_to_numpy=True)
print(f"✅ Created embeddings of shape: {embeddings.shape}")

✅ Created embeddings of shape: (3, 384)


In [13]:
import chromadb
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="pdf_embeddings")
collection.add(
ids=[f"chunk_{i}" for i in range(len(texts))],
documents=texts,
embeddings=embeddings.tolist()
)
print(f"✅ Saved {len(texts)} chunks into ChromaDB")

✅ Saved 3 chunks into ChromaDB


In [36]:
query = "Give me education details of revati"

In [37]:
query_embedding = model.encode([query], convert_to_numpy=True)
results = collection.query(query_embeddings=query_embedding.tolist(), n_results=5)

In [38]:
if results["documents"]:
    print("\n🔎 Retrieved Chunks:")
for i, doc in enumerate(results["documents"][0]):
    print(f"{i+1}. {doc}\n")


🔎 Retrieved Chunks:
1. Revati  Patil  MCA  graduate  with  strong  knowledge  of  frontend  and  backend  development,  eager  to  apply  
technical
 
skills
 
to
 
real-world
 
projects.
 
 revatip290@gmail.com   +91  9579412102  g i t h u b . c o m / R e v a t i v P a t i l   l i n k e d i n . c o m / i n / r e v a t i - v - p a t i l /  
WORK  EXPERIENCE Humming  Byte  Pvt.Ltd Web  Development  Intern   June  2023  -  Present •  Developed  and  maintained  responsive  full-stack  web  applications  using  Node.js ,  React.js ,  and  modern  
frameworks.
 •  Integrated  Firebase  backend  services,  implementing  secure  authentication  and  real-time  database  
functionalities.
 •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  performance  to  improve  application  eﬃciency  by  
reducing
 
response
 
time.

2. functionalities.
 •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  performance  to  improve  application  eﬃciency  by  
reducing
 
response
 
time.
 

In [39]:
print(results)

{'ids': [['chunk_0', 'chunk_1', 'chunk_2']], 'embeddings': None, 'documents': [['Revati  Patil  MCA  graduate  with  strong  knowledge  of  frontend  and  backend  development,  eager  to  apply  \ntechnical\n \nskills\n \nto\n \nreal-world\n \nprojects.\n \n revatip290@gmail.com   +91  9579412102  g i t h u b . c o m / R e v a t i v P a t i l   l i n k e d i n . c o m / i n / r e v a t i - v - p a t i l /  \nWORK  EXPERIENCE Humming  Byte  Pvt.Ltd Web  Development  Intern   June  2023  -  Present •  Developed  and  maintained  responsive  full-stack  web  applications  using  Node.js ,  React.js ,  and  modern  \nframeworks.\n •  Integrated  Firebase  backend  services,  implementing  secure  authentication  and  real-time  database  \nfunctionalities.\n •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  performance  to  improve  application  eﬃciency  by  \nreducing\n \nresponse\n \ntime.', 'functionalities.\n •  Enhanced  UI/UX  with  CSS ,  and  optimized  REST  API  perfo

In [40]:
joined_text = "\n\n".join(results['documents'][0])
prompt = f"Please rephrase the following text to be a clear, concise answer:\n\n{joined_text}"

In [41]:
from google import genai

In [42]:
from google import genai
import os


api_key = os.environ.get("GENAI_API_KEY")
if not api_key:
# Fallback: manually paste your key here if env variable is not set
    api_key = "AIzaSyGtupu_HpFO2OUudVL1x-HYsJBRX1NxRE" # <-- replace this only for quick testing


if not api_key or api_key.startswith("PASTE"):
    raise ValueError("No valid API key found. Set GENAI_API_KEY environment variable or paste key above.")


client = genai.Client(api_key=api_key)

In [ ]:
response = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
if response:
    print("\n🤖 Final Answer:\n", response.text)


🤖 Final Answer:
 Revati Patil is a recent MCA graduate with strong full-stack development skills (Node.js, React.js, MongoDB) and experience building web applications and Java-based desktop applications. She interned at Humming Byte Pvt. Ltd., where she developed and maintained full-stack web applications, integrated Firebase backend services, and optimized REST API performance. She also has project experience in developing management systems using Java and full-stack web technologies. (Contact: revatip290@gmail.com, +91 9579412102, github.com/RevatiVPatil, linkedin.com/in/revati-v-patil/)

